In [1]:
# pip3 install mariadb
# pip3 install pandas
# pip3 install numpy  
# sudo apt-get install libatlas-base-dev
# pip3 install matplotlib
# pip3 install ipympl
# pip3 install ipywidgets

%matplotlib widget
%config InlineBackend.figure_format = 'png'

import os
import sys
import mariadb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, interactive_output, fixed, interact_manual, IntSlider, IntRangeSlider, HBox
import ipywidgets as widgets

In [2]:
username = os.environ.get("db_user")
password = os.environ.get("db_pwd")

def update_db_name(change):
    #print(change.new)
    global db_name
    db_name = change.new
    
list_db_name = widgets.RadioButtons(
        options=['reformer', 'catalyst', 'Reformer_BW', 'Reformer_SE'],
        description='Select Database:',
        disabled=False
        )
db_name = list_db_name.value
list_db_name.observe(update_db_name, 'value')
display(list_db_name)

RadioButtons(description='Select Database:', options=('reformer', 'catalyst', 'Reformer_BW', 'Reformer_SE'), v…

In [12]:
try:
    # Connect to MariaDB Platform
    conn = mariadb.connect(
        user=username,
        password=password,
        host="localhost",
        port=3306,
        database=db_name,
        autocommit=True
    )

    # Get Cursor for tx
    cur = conn.cursor(named_tuple=False)
    #print(conn)
    print(cur)
    print(f"Connected to {db_name}@localhost ")
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)

Connected to reformer@localhost 


In [66]:
cur.execute('SHOW tables')
table_dropdown = widgets.Dropdown(options=[u for i in cur.fetchall()[:] for u in i][::-1], description='Database')
cur.execute(f"SELECT * FROM {table_dropdown.value}")
cur.fetchall()
time_slider = widgets.IntRangeSlider(value=[0, cur.rowcount], min=0, max=cur.rowcount, step=1200, description='Time Range', continuous_update=False)
checkbox = widgets.Checkbox(value=False, description='Steady-State Only', disabled=False, indent=False)

def update_time_range(*args):
    cur.execute(f"SELECT * FROM {table_dropdown.value}")
    cur.fetchall()
    time_slider.max = cur.rowcount
table_dropdown.observe(update_time_range, 'value')

In [67]:
def plot(Table_name, Time, SS):
    print(Table_name, Time)
    #plt.clf()
    fig, (ax_TC, ax_DFM, ax_GA) = plt.subplots(3, constrained_layout=True, figsize=(10, 10), sharex=False)
    fig.canvas.toolbar_position = 'left'
    
    cur.execute(f"SELECT * FROM {Table_name}")
    df = pd.DataFrame(cur.fetchall(), columns=[entry[0] for entry in cur.description]).set_index('Id')
    
    if SS:
        ls=[]
        for i in Set_Point_lst:
            i.cond(df, TC='TC10', Scale='Scale')
            i.eff_calc(df, Scale='Scale', DFM_RichGas='DFM_RichGas', GA_H2='GA_H2')
            if i.ss_time:
                for u in i.ss_time:
                    #print(u)
                    for v in range(u[0],u[-1]):
                        ls.append(v)
            #print(ls)
        _bmask = pd.Series([True if i in ls else False for i in range(0,len(df))])
        df = df.where(_bmask, 0)
    
    df_GA = pd.DataFrame({
        'GA_H2':df['GA_H2'],
        'GA_CO':df['GA_CO'],
        'GA_CO2':df['GA_CO2'],
        'GA_CH4':df['GA_CH4'],
        'GA_N2':df['GA_N2'],
    }, index=df.index)
    df_DFM = pd.DataFrame({
        'DFM_AOG':df['DFM_AOG'],
        'DFM_RichGas':df['DFM_RichGas'],
    }, index=df.index)
    
    df['TC10'].plot(legend=True, ax=ax_TC, kind='area', color='lightblue')
    df['TC6'].plot(legend=True, ax=ax_TC)
    df['TC7'].plot(legend=True, ax=ax_TC)
    df['TC8'].plot(legend=True, ax=ax_TC)
    df['TC9'].plot(legend=True, ax=ax_TC)
    df['TC11'].plot(legend=True, ax=ax_TC)
    df['EVA_Out'].plot(legend=True, ax=ax_TC,
                       title=f'TC_{Table_name}', 
                       ylabel='Temp[oC]', 
                       ylim=(0,600), 
                       yticks=range(0,600,100),
                      )
    ax_TC_2 = ax_TC.twinx()
    ax_TC_2.set_ylim([0, 600])
    ax_TC_2.set_yticks(range(0,600,100)) 
    
    df_DFM.plot(legend=True, ax=ax_DFM, kind='area', stacked=True, 
            title=f'DFM_{Table_name}',
            ylabel='Gas Production [LPM]', 
            #grid=True,  
            ylim=(0,140), 
            yticks=range(0,140,10)
           )
    ax_DFM_2 = df['Scale'].plot(legend=True, ax=ax_DFM, secondary_y=True)
    ax_DFM_2.set_ylabel('Scale [g/min]')
    ax_DFM_2.set_ylim(0,100)
    
    df_GA.plot(legend=True, ax=ax_GA, kind='area', stacked=True, 
            title=f'GA_{Table_name}', 
            ylabel='Gas Composition [%]', 
            #grid=True,  
            xlim=Time, 
            xticks=range(Time[0],Time[1],1200),
            ylim=(0,100), 
            yticks=range(0,100,10)
           )
    df['CO_line'] = df['GA_CO']
    ax_GA_2 = df['CO_line'].plot(legend=True, ax=ax_GA, secondary_y=True)
    ax_GA_2.set_ylabel('CO [%]')
    ax_GA_2.set_ylim(0,5)
    
    plt.show()

In [68]:
class SerEnergy_Set_Point:
    density_H2 = 0.08988 # kg/m3 @ STP
    enthalpy_H2 = 120.21 # MJ/kg-LHV
    enthalpy_MeOH = 20.09 # MJ/kg-LHV
    MeOH_weight = 0.543 # 54.3wt%
    cont_step = 60 #continue for 1 min
    TC_range = 5 #5oC
    Scale_range = 5 #5g/min
    continuity = 10 #10min
    def __init__(self, temp=0, weight_rate=0, dummy=0):
        # set points
        self.temp = temp 
        self.weight_rate = weight_rate
        self.dummy = dummy
        
        # steady-state
        self.ss_time = []
        #self.ss_temp = 0 # avg
        self.ss_weight_rate = [] #avg
        self.ss_DFM_RichGas = [] # avg
        self.ss_GA_H2 = [] # avg
        #self.ss_DFM_AOG = []
        self.heff = []
        
    def cond(self, df, TC='', Scale=''):
        self.ss_time = []
        count_60 = 0 #counter for steady mins
        ss_temp = []
        ss_lst = []
        ss = ((df[TC] >= (self.temp - self.TC_range)) & (df[TC] <= (self.temp + self.TC_range))) \
            & ((df[Scale] >= (self.weight_rate - self.Scale_range)) & (df[Scale] <= (self.weight_rate + self.Scale_range)))
        #print(ss.sum())
        for i in range(0, len(ss), self.cont_step):
            if i + self.cont_step <= len(ss):
                if ss.iloc[i:i + self.cont_step].sum() == 60:
                    count_60 += 1
                    ss_temp.append(i)
                else:
                    count_60 = 0
                    if ss_temp != []:
                        ss_lst.append(ss_temp)
                        ss_temp = []
                    continue
        #print(ss_lst)
        for u in ss_lst:
            if len(u) >= self.continuity:
                self.ss_time.append((u[0], u[-1]))
    def eff_calc(self, df, Scale='', DFM_RichGas='', GA_H2=''):
        if self.ss_time:
            self.ss_weight_rate = []
            self.ss_DFM_RichGas = []
            self.ss_GA_H2 = []
            self.heff = []
            for u in self.ss_time:
                #print(u)
                ls = [v for v in range(u[0],u[-1])]
                df_mean = df[df.index.isin(ls)]
                self.ss_weight_rate.append(df_mean[Scale].mean())
                self.ss_DFM_RichGas.append(df_mean[DFM_RichGas].mean())
                self.ss_GA_H2.append(df_mean[GA_H2].mean())
            # should apply numpy...
            H2_prod = (self.ss_DFM_RichGas[0] * self.ss_GA_H2[0] / 100) - self.dummy # LPM
            #print(H2_prod)
            H2_prod = H2_prod * self.density_H2 / 1000 # kg/min @ STP
            #print(H2_prod)
            self.heff.append((H2_prod * self.enthalpy_H2) / (self.ss_weight_rate[0] / 1000 * self.MeOH_weight * self.enthalpy_MeOH) * 100) # enthalpy eff [%]
        else:
            pass
        #print(self.ss_weight_rate, self.ss_DFM_RichGas, self.ss_GA_H2)
        print(self.heff)
        

In [69]:
Set_Point_lst = [SerEnergy_Set_Point(temp=300, weight_rate=18.91), 
                 SerEnergy_Set_Point(temp=305, weight_rate=31.85), 
                 SerEnergy_Set_Point(temp=315, weight_rate=49.02, dummy=9.5), 
                 SerEnergy_Set_Point(temp=325, weight_rate=63.1), 
                 SerEnergy_Set_Point(temp=332, weight_rate=81.45, dummy=12.5)]

plot = interactive_output(plot, {'Table_name':table_dropdown, 'Time':time_slider, 'SS':checkbox})
control = HBox([table_dropdown, time_slider, checkbox])
widgets.VBox([control, plot])

In [70]:
cur.execute("SELECT * FROM platform_2021_12_29_10_21")
df = pd.DataFrame(cur.fetchall(), columns=[entry[0] for entry in cur.description]).set_index('Id')

for i in Set_Point_lst:
    i.eff_calc(df, Scale='Scale', DFM_RichGas='DFM_RichGas', GA_H2='GA_H2')

[]
[]
[]
[]
[]


In [ ]:
conn.close()